# Quick and dirty test of rough draft of SOMPZInformer

In [ ]:
# usual imports
import os
import numpy as np
from rail.core.utils import RAILDIR
#from rail.estimation.algos.sompz_version.utils import RAIL_SOMPZ_DIR
#from rail.pipelines.estimation.inform_all import InformPipeline
from rail.core import common_params
import matplotlib.pyplot as plt

In [ ]:
from rail.estimation.algos.sompz import SOMPZInformer

In [ ]:
from rail.core.data import TableHandle
from rail.core.stage import RailStage

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

read in some test data for the "wide" som, it's just 20,000 mock ugrizy magnitudes and errors from cosmodc2

In [ ]:
from rail.core.utils import find_rail_file
#trainFile = find_rail_file('examples_data/testdata/test_dc2_training_9816.hdf5')
testFile = find_rail_file('examples_data/testdata/test_dc2_validation_9816.hdf5')
#deep_data = DS.read_file("training_data", TableHandle, trainFile)
wide_data = DS.read_file("input_wide_data", TableHandle, testFile)

I have copied the file that Justin made to this directory and will read that fits file in to test the "deep" som.  If you have the file somewhere else, then update the "hsc_file" path in the cell below:

In [ ]:
hsc_file = "hsc_deep_incl_nir_incl_specz_TEST_ONLY_NO_SCIENCE_2023-10-18.fits"
deep_data = DS.read_file("training_data", TableHandle, hsc_file)

To speed things up a bit, I'm going to grab 10% of the data and actually use that.  It should work with the big data, but this will be faster to train up the som for a quick demo:

In [ ]:
subset = deep_data()[::10]

In [ ]:
smaller_data = DS.add_data("smaller_data", subset, TableHandle)

create a few lists of names that we'll need for the names of the input columns that we'll used in the SOM, along with their errors.

In [ ]:
bands = ['u','g','r','i','z','J','H', 'K']
deepbands = []
deeperrs = []
zeropts = []
for band in bands:
    deepbands.append(f'flux_DES_DEEP_{band}')
    deeperrs.append(f'flux_err_DES_DEEP_{band}')
    zeropts.append(30.)

In [ ]:
deepbands

make a dictionary of the parameters that we'll feed into the informer for the deep som.  Because we're using HSC data we need the non-default names fo the input columns (`inputs_deep`) and the errors (`input_errs_deep`).  We'll feed in a list for the zero points (`zero_points`) as well, as eventually we'll want to add a check to make sure all three of those have the same length:

In [ ]:
som_params = dict(inputs_deep=deepbands, input_errs_deep=deeperrs, zero_points_deep=zeropts, 
                  convert_to_flux=False, set_threshold=True)

deep_groupname and hdf5_groupname (along with an unused wide_groupname) have to do with hdf5 groups and how we read the data, hdf5_groupname is not used in this stage, but it is expected by the parent class, so I'm setting it here:

In [ ]:
som_inform = SOMPZInformer.make_stage(name="som_informer", deep_groupname='', hdf5_groupname='',
                                      model="TEST_HSC_DC2_model.pkl", **som_params)

Now, run the informer:

In [ ]:
som_inform.inform(smaller_data, wide_data)

Let's look at the results by reading the model we just wrote out, `TEST_HSC_DC2_model.pkl` and looking at what it contains.  It should be a dictionary that contains two soms: `deep_som` and `wide_som`, along with the `deep_columns`, `wide_columns`, `deep_err_columns`, and `wide_err_columns` lists.  I think we'll want to store these as they basically define the ordering of the columns and errors, and we'll want that the same for data that we pass in (if we pass in more data?  I need to look at how the estimate stage works...)

In [ ]:
import pickle

In [ ]:
with open("TEST_HSC_DC2_model.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
model

In [ ]:
import rail.estimation.algos.som as SOMFUNCS

In [ ]:
SOMFUNCS.somDomainColorsnok(model['deep_som'])

In [ ]:
SOMFUNCS.somDomainColors(model['deep_som'])

In [ ]:
SOMFUNCS.somDomainColorsnok(model['wide_som'])

In [ ]:
SOMFUNCS.somPlot2d(model['deep_som'])